In [168]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")
import urllib2
import re
from subprocess import PIPE, Popen
import os
import glob
import gzip
from itertools import islice
import ftplib
import math
from sklearn import feature_selection
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from scipy.stats.stats import pearsonr
import random
from sklearn import linear_model

For this study, methlyation datasets were obtained from Gene Expression Omnibus (GEO) database using GSE identifiers. 
File "geo-noncancer-datasets.txt" and "geo-cancer-datasets.txt" files provides an overview of the publicly available human DNAm data sets used in this study for cancer and non-cancer cases, respectively.  

Some sections of the code below will only run one GSE id for demo purpose. 

All the raw data and processed will be found here:
https://drive.google.com/folderview?id=0B5edDxPaIzYHQnlWR3lGMFYtOUk&usp=sharing

In [143]:
geo_noncancer_list = []
#read the non-cancer dataset list from tab delim file
noncancer_df=pd.read_csv('geo-noncancer-datasets.txt',sep='\t')
#extract the geo identifiers
geo_noncancer_list = list(set(noncancer_df.Availability)) 
print "Non-cancer datasets",len(geo_noncancer_list)

#read the non-cancer dataset list from tab delim file
cancer_df=pd.read_csv('geo-cancer-datasets.txt',sep='\t')
#extract the geo identifiers
geo_cancer_list = (cancer_df.Availability).values
print "Cancer datasets", len(geo_cancer_list)

Non-cancer datasets 48
Cancer datasets 7


DNA methylation relevant data was available in different files. 

For each GSE identifier there were Series matrix files  - containing expriment metadata like tissue source, chronogical age and experiment id's. 
Signal files -  These contained methlyation signals for every biomarker  as methylated and non-methylated, from which beta values were calculated. In some cases beta values were already made available. 

Data format challenges - There were many challenges in terms dowloading required files as there were no standard naming conventions and in some cases internal format was also quite different. Also, in some cases methlyation signals were available throgh series files instead of signal files.

Some of the edge cases had to be manually converted into a standard format in order for code to work.  

In [144]:
#ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE40nnn/GSE40700/matrix/GSE40700_series_matrix.txt.gz
#below functions check whether a ftp works for gse id according to standard format and downloads them 
#Note: download commands, adding a break in for loop for demo puropose, so that it shows working for one gseid
def check_urllink(ftplink):
    #check if the ftp link exists and check for edge cases 
    try:
        urllib2.urlopen(ftplink)
        return ftplink
    except urllib2.HTTPError, e:
        print(e.code)
        return 1
    except urllib2.URLError, e:
        print(e.args)
        return 1
    
#iterate over the accessions to get series file for metadata
#(these contain methylation signals in some cases)
def fetch_series_data(geo_noncancer_list):
    outdir ='data/geo-series'
    gse_noncancer_edge=[]
    failed_fetch_list = []
    for i in geo_noncancer_list:
        key_num = i.split("GSE")[1]
        url_key= "GSE"+key_num[0:2]
        file_n = i + "_series_matrix.txt.gz"
        url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/"+url_key.replace(" ","")+ "nnn/" + i + "/matrix/" 
        ftplink= (url + file_n).replace(" ","")
        result = check_urllink(ftplink)
        ftp_cmd= "cd " + outdir + "; ftp " + ftplink + " ;cd -"
        if result == 1:
            print "link failure, cannot find series files", ftplink
            gse_noncancer_edge.append(i)
        else:
            print "link Success: Fetching ", ftplink
            #below part triggers ftp download cmd in shell using subprocess
            pfetch = Popen(ftp_cmd,stdout=PIPE,stderr=PIPE,shell=True,close_fds=True)
            std_out, std_err = pfetch.communicate()
            exit_code = pfetch.returncode
            if exit_code:
                failed_fetch_list.append(i)
            #os.system(ftp_cmd)
        #adding break here, just so that it works for one accession
        break
    return gse_noncancer_edge, failed_fetch_list 


In [145]:
#download noncancer datasets
gse_noncancer_edge, failed_fetch_list = fetch_series_data(geo_noncancer_list)

link Success: Fetching  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE28nnn/GSE28746/matrix/GSE28746_series_matrix.txt.gz


In [146]:
#download series files for cancer datasets
gse_cancer_edge, failed_fetch_list = fetch_series_data(geo_cancer_list )

link Success: Fetching  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE31nnn/GSE31979/matrix/GSE31979_series_matrix.txt.gz


In [122]:
print "number of GEO non-cancer edge cases", len(gse_noncancer_edge)
print "number of failed fetches for normal GEO data links" , len(failed_fetch_list)

number of GEO non-cancer edge cases 0
number of failed fetches for normal GEO data links 0


Earlier we downloaded series files (~6gb compressed series files), now below functions will get signal files (when done these were ~5gb compressed), 
signal files contain methylation and non methylation signals, which have to be converted into beta values
More about betavalues in methylation can be found here: http://www.ncbi.nlm.nih.gov/pmc/articles/PMC3012676/

In [149]:
def ftp_listfiles(ftplink):
    filelist = []
    urlpath = urllib2.urlopen(ftplink)
    string = urlpath.read().decode('utf-8').encode()
    for txt in string.split("\n"):
        f = txt.split(" ")[-1]
        if re.search("signal", f) or re.search("Signal", f):
            filelist.append(f)
        elif re.search("normalize",f) or re.search("Normalize",f) :
            filelist.append(f)
    return filelist

def fetch_signalfiles(geo_list):
    gse_edge=[]
    failed_fetch_list = []
    outdir = 'data/geo-signals'
    for i in geo_list:
        i = i.replace(" ",'')
        key_num = i.split("GSE")[1]
        url_key= "GSE"+key_num[0:2]
        url = "ftp://ftp.ncbi.nlm.nih.gov/geo/series/"+url_key+ "nnn/" + i + "/suppl/"
        files = ftp_listfiles(url)
        for file_n in files:
            ftplink= (url + file_n).replace(" ","").replace("\r",'')
            result=check_urllink(ftplink)
            ftp_cmd= "cd " + outdir + "; ftp " + ftplink + " ;cd -"
            if result == 1:
                print "link failure, cannot find signal file", ftplink
                gse_edge.append(i)
            else:
                print "Link Success: Fetching ", ftplink
                pfetch = Popen(ftp_cmd,stdout=PIPE,stderr=PIPE,shell=True,close_fds=True)
                std_out, std_err = pfetch.communicate()
                exit_code = pfetch.returncode
                if exit_code:
                    failed_fetch_list.append(i)
                #os.system(ftp_cmd)
            #adding break for demo purpose
        break
    return gse_edge, failed_fetch_list  

def beta_val(df):
    #Max(M,0)/[Max(M,0)+Max(U,0)+100]. 
    #beta val = methylated signal /(unmethylated signal + methylated signal + 100)
    #Thus, β values range from 0 (completely un-methylated) to 1 (completely methylated) . 
    return (float(df[1])/float(df[1] + df[0]+100))

In [150]:
signal_resem, failed = fetch_signalfiles(geo_noncancer_list)
len (signal_resem), len(failed)

Link Success: Fetching  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE28nnn/GSE28746/suppl/GSE28746_non-normalized.txt.gz


(0, 0)

In [151]:
signal_resem, failed = fetch_signalfiles(geo_cancer_list)
len (signal_resem), len(failed)

Link Success: Fetching  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE31nnn/GSE31979/suppl/GSE31979_matrix_signal_intensities.txt.gz


(0, 0)

Since the signal files were really big, I had to chunk them and read it sequentially processed them and append. Below functions work on chunk dataframes and return beta values. There is lot of scope for optimization by using spark parallel processing.  But decided to try optimizations later and make sure the method works first. 

In [153]:
def open_signalfile(file1):
    #accepts file and returns file chunk iterator
    signaldf_iter = None
    signaldf_iter=pd.read_csv(file1,sep="\t",comment="#",index_col=0,iterator=True, chunksize=10000)
    return signaldf_iter

def process_chunkdf(chunk):
    #for each chunk df calculates the beta values 
    betadf = pd.DataFrame()
    #print chunk.shape
    filter = []
    #dropping rows if any of them have NaN
    chunk.dropna(axis=0, how='any', thresh=None, subset=None, inplace=True)
    list_cols = chunk.columns
    #adding edge cases
    #this takes care of remvoing unwanted columns to reduce memory usage
    filter = ~(list_cols.str.contains("pval|background|probeid|symbol",case=False, regex=True))
    filter_cols = list_cols[filter]
    chunk=chunk[filter_cols]
    #print chunk.shape 
    #further check if the dataframe has beta values already.
    filter = filter_cols.str.contains("beta",case=False, regex=True)
    #if the file contain beta values already calculated, then fetch only those columns
    if np.sum(filter) > 0:
        filter_cols = filter_cols[filter]
        betadf=chunk[filter_cols]
        #rename index for consistency 
        betadf.index.names = ['cpg_id'] 
    else:
        #if file contains methlyated and unmethlyated then calculate beta values.
        chunk.index.names = ['cpg_id'] 
        for i in range(len(chunk.columns))[::2]:
            col_1 = chunk.columns[i]
            col_2 = chunk.columns[i+1]
            #print col_1 ,col_2
            #create column with polished name
            col_new = col_1.replace('Unmethylated Signal','').replace('Signal_A','')
            betadf[col_new] = chunk[[col_1,col_2]].apply(beta_val,axis=1)
    return betadf

def create_betasig_df(iterator):
    #takes file chunk iterater and returns betavalues df for full file
    finaldf = pd.DataFrame()
    i = 0
    for chunk in iterator:
        i= i +1
        #print chunk.shape
        betadf = process_chunkdf(chunk)
        finaldf = pd.concat([finaldf, betadf])
    return finaldf

In [154]:
def get_signal_methyldf(filearr):
    #iterates over signal files , get chunk iterators , beta values and writes them out to file
    outputpath = 'data/geo-betasignals/'
    trouble_samples = []
    for file1 in filearr:
        print file1
        tmpdf=pd.DataFrame()
        signaldf_iter = ''
        try:
            signaldf_iter = open_signalfile(file1)
            tmpdf = create_betasig_df(signaldf_iter)
        except Exception as e:
            trouble_samples.append(file1)
            continue
        #create output file name for betavalues
        path,filename = os.path.split(file1)
        filename, ext = os.path.splitext(filename)
        filename, ext = os.path.splitext(filename)
        tmpdf.to_csv(outputpath+filename+'.csv',sep=',')
    return trouble_samples

Calling the beta value calculation functionality on each files and capture edge cases as trouble_samples. 

In [156]:
%%time
#demo block
filearr= glob.glob('data/geo-signals/'+'*.txt.gz')
trouble_samples = get_signal_methyldf(filearr)
print len(trouble_samples)

data/geo-signals/GSE28746_non-normalized.txt.gz
data/geo-signals/GSE31979_matrix_signal_intensities.txt.gz
0
CPU times: user 6min 37s, sys: 5.85 s, total: 6min 43s
Wall time: 6min 43s


In [158]:
#full run, commenting out to avoid recalculation
filearr= glob.glob('data/geo-signals/'+'*.txt.gz')
#trouble_samples = get_signal_methyldf(filearr)
edge_case = ['GSE20236','GSE26033','GSE30090','GSE30780','GSE32146','GSE34257','GSE34639','GSE36064','GSE43269']
len(edge_case)

9

Had to manually curate the edge case signal files as the column structure was random.
There was no consistency or pattern among these to customize the code, moved them under "geo-nonuniform-signalfiles" dir and processed files to have - cpgid, non-methylation column, methylation column, pval(optional). 
And, repeated the beta val calculations

In [8]:
trouble_samples=glob.glob('data/geo-nonuniform-signalfiles/*.txt.gz')
#commenting out to avoid recalcualtion
#trouble_samples = get_signal_methyldf(trouble_samples)

some of the accessions did not have signal files, instead series files contained the beta values. 
We will deal with them later. Below function takes care of just reading series file into df.

In [160]:
def create_series_methyldf(file1):
    #read series file, skip comment line and return the df as methylation df
    maindf=pd.read_csv(file1,comment="!",sep="\t",index_col=0,skip_blank_lines=True)
    return maindf

In [162]:
gse_without_signals=['GSE17448','GSE20067','GSE30758','GSE32393','GSE41169','GSE42510','GSE40700','GSE32149','GSE19711','GSE20712','GSE25062']
for i in gse_without_signals:
    arr = glob.glob( 'data/geo-series/'+ i+ "*.txt.gz")
    for file1 in arr:
        tmpdf = create_series_methyldf(file1)
        path,filename = os.path.split(file1)
        filename, ext = os.path.splitext(filename)
        filename, ext = os.path.splitext(filename)
        tmpdf.to_csv(outputpath+filename+'.csv',sep=',')

Below section takes care of downloading metadata from series files for every sample
Metadata includes : chronological age, gender , tissue source, sample GEO id, sample id and if cancer data

In [163]:
def create_metadata(file1,cancer=0):
    #approx number of lines to fetch tissue information
    N = 2000
    f = gzip.open(file1, 'rb')
    #Reading first 2000 lines, header lines are inconsistent but cannot contain such large number. 
    headerlines = islice(f, N)
    metadict = {'sample_geo_id':[],'age':[],'gender':[],'sample_id':[],'cancer':[]}
    for i in headerlines:
        line = i.strip("\n")
        if re.search('!Sample_geo_accession',line,re.IGNORECASE):
            metadict["sample_geo_id"] = line.replace('"','').split("\t")[1:]
            ref_len = len( metadict["sample_geo_id"] )
            #intialize metadata array lengths according to geo ref lengths
            #metadict["sample_id"] = metadict["sample_geo_id"]
            metadict["tissue_src"] = [np.nan]*ref_len
            metadict["age"] = [np.nan]*ref_len
            metadict["gender"] = [np.nan]*ref_len
            metadict["cancer"] = [cancer]*ref_len
        if re.search('!Sample_source_name_ch1',line,re.IGNORECASE):
            metadict["tissue_src"] = line.replace('"','').split("\t")[1:]
        if re.search('!Sample_title',line,re.IGNORECASE) :
            metadict["sample_id"] = line.replace('"','').split("\t")[1:]
        if re.search('!Sample_characteristics_ch1',line, re.IGNORECASE):
            #iterate over each line characteristics line to fetch age, gender info
            temp_arr = line.split("\t")[1:]
            for l in range(0,len(temp_arr)):
                ele = temp_arr[l]
                if re.search('ageatdiagnosis',ele, re.IGNORECASE) or re.search('"age',ele, re.IGNORECASE):
                    #check for months , default is years
                    ele = ele.replace('"','')
                    years = re.sub("[^0-9]", "", ele)
                    if re.search('month',ele,re.IGNORECASE):
                        years = float(years)/12.0
                    metadict["age"][l] = years
                if re.search('sex', ele, re.IGNORECASE) or re.search('gender', ele, re.IGNORECASE):
                    metadict["gender"][l] = ele.replace('gender','').replace('sex','').replace(':','').replace(' ','').replace('"','')
    metadatadf = pd.DataFrame(metadict)
    return metadatadf

In [165]:
%%time
datasets = glob.glob("data/geo-series/*txt.gz")
outputpath = 'data/geo-metadata/'
for file1 in datasets:
    #create output file name
    path,filename = os.path.split(file1)
    filename, ext = os.path.splitext(filename)
    filename, ext = os.path.splitext(filename)
    #check if the dataset is from cancer samples
    if (filename.split("_")[0]).replace("\t",'') in geo_cancer_list:
        flag= 1
    else:
        flag = 0
    metadf = create_metadata(file1,cancer=flag)
    metadf.to_csv(outputpath+filename+'_meta.csv',sep=',',index=False)

CPU times: user 445 ms, sys: 9.51 ms, total: 455 ms
Wall time: 475 ms


Normalization :

Since datasets were obtained from different platforms, it was curical for us to normalize the data for signals to be comparable. The Illumina 450K platforms uses two different chemical assays. We used R method from Teschendorff et al 2013 based on a intra-array normalization strategy for the 450K platform, called BMIQ (Beta MIxture Quantile dilation), which adjusts beta-values of type II probes into a statistical distribution characteristic of type I probes.  

Reference for normalization : Teschendorff AE, Marabita F, Lechner M, Bartlett T, Tegner J, Gomez-Cabrero D, Beck S: A beta-mixture quantile normalization method for correcting probe design bias in Illumina Infinium 450 k DNA methylation data. Bioinformatics 2013, 29:189-196.
http://www.ncbi.nlm.nih.gov/pubmed/23175756

In [166]:
normfailed=[]
def normalize_datasets(file1):
        outputpath = 'data/geo-normalized/'
        probeannotfile = 'probeAnnotation21kdatMethUsed.csv'
        #create output file name for betavalues
        path,filename = os.path.split(file1)
        filename, ext = os.path.splitext(filename)
        filename, ext = os.path.splitext(filename)
        outfile = outputpath + "/" + filename +"_norm.csv"
        norm_cmd = "Rscript  Normalization.R " + file1 + " " + probeannotfile + " " + outfile
        pnorm = Popen(norm_cmd,stdout=PIPE,stderr=PIPE,shell=True,close_fds=True)
        std_out, std_err = pnorm.communicate()
        exit_code = pnorm.returncode
        if exit_code:
            print "normalization error", file1
            normfailed.append(file1+ ":"+norm_cmd )
        return normfailed        

In [169]:
%%time
filearr= glob.glob('data/geo-betasignals/'+'*.csv')
for file1 in sorted(filearr):
    path,f = os.path.split(file1)
    failnorm_files = normalize_datasets(file1)
    #adding break to iterate over only one file for demo
    break

normalization error data/geo-betasignals/GSE28746_non-normalized.csv
CPU times: user 1.67 ms, sys: 5.55 ms, total: 7.22 ms
Wall time: 5.06 s


For our studies only 21k probes will retained and sample id will be replaced with geo accessions.
Also, age, tissue_src and other metadata will be added to normalized dataframe to final df

In [174]:
#extract cpgid for 21k probes
probdf=pd.read_csv('probeAnnotation21kdatMethUsed.csv')
#extract tissue information for all geo-accessions, cancer and non-cancer
standard_tissuedf = pd.read_csv("geo-accession-tissueinfo.txt", sep="\t")
#reading the file for tissue info for each geo accession
tissue_list = list(standard_tissuedf["Availability"].values)
#fetch normalized files 
normalized_files = glob.glob('data/geo-normalized/*_norm.csv')
metadir = 'data/geo-metadata/'
preprocess_outdir = 'data/geo-preprocessed/'
for file1 in normalized_files:
    """iterate over normalized files and based on filename accession
    fetch metadata and merge them. Since tissue_src line is too lengthy
    we will use shortened versions which we read from standard-tissuedf"""
    try:
        path,filename = os.path.split(file1)
        gse_id = filename.split("_")[0]
        metafile = glob.glob(metadir + '/' + gse_id + "*")[0] 
        metadf = pd.read_csv(metafile)
        #get standard tissue name and replace in metadf 
        #removing the tissue_src information with standard tissue notations. 
        #for example  : "uterine cervix" instead of "genomic dna from uterine cervix"
        if gse_id in tissue_list:
            idx = tissue_list.index(gse_id)
            tissue_name = standard_tissuedf["DNA origin"][idx]   
            metadf["tissue_src"] = [tissue_name] * metadf.shape[0]
        print tissue_name
        normdf = pd.read_csv(file1,index_col=0)
        normdf.reset_index(inplace=True)
        normdf.set_index("index",inplace=True)
        normdf.columns = list(probdf["Name"])
        normdf.reset_index(inplace=True)
        if re.search("GS",normdf["index"][0]):
            normdf.rename(columns={"index":"sample_geo_id"}, inplace=True)
        else:
            normdf.rename(columns={"index":"sample_id"}, inplace=True)
        resultdf = pd.concat([normdf, metadf], axis=1, join_axes=[normdf.index])
        resultdf = resultdf.T.groupby(level=0).first().T
        resultdf.to_csv(preprocess_outdir+filename.replace("_norm.csv","_final.csv.gz"),sep=',', index=False)
    except Exception as e:
        print "verify data", file1
        print "exception ", e

verify data data/geo-normalized/GSE28746_non-normalized_norm.csv
exception  list index out of range
Breast


In [172]:
standard_tissuedf["DNA origin"].values

array(['MSC (bonemarrow)', 'Blood WB', 'Blood WB', 'Blood WB', 'Blood WB',
       'Buccal', 'Blood CD4+CD14', 'Dermal fibroblast', 'Buccal', 'Heart',
       'Chimp+Human Tissues', 'Prostate NL', 'Sperm', 'Blood PBMC',
       'Blood Cord', 'Saliva', 'HematopoieticStemAndNormalPrimaryTissue',
       'hESC                                      ', 'Gastric',
       'Stem cells+Somatic Cells', 'Uterine Cervix', 'Blood PBMC',
       'Stem cells+Somatic Cells', 'Colon', 'Blood PBMC', 'Breast NL',
       'Saliva', 'Saliva', 'Blood Cord', 'Blood CD4 Tcells',
       'Blood Cell Types', 'Blood PBMC', 'Muscle', 'Blood Cord',
       'Blood PBMC', 'Reprogrammed mesenchymal stromal cells ', 'Liver ',
       'Fat Adip', 'Muscle', 'Brain Cerebellar', 'Brain Occipital Cortex',
       'Brain CRBLM', 'Various Tissues', 'Blood WB', 'Blood WB', 'Ape WB',
       'BrainVariousCells', 'Heart', 'Buccal', 'Buccal',
       'Blood Cell Types', 'Cartilage Knee', 'Placenta', 'Sperm', 'Brain',
       'Breast', 'Colore